# Pandas basics

- series
- dataframes:
    - creating, reading and general df infos
    - selecting

In [2]:
import pandas as pd

### Series

In [3]:
# Series : can be seen as list 
pd.Series([1, 2, 3, 4, 5], index=["First","Second","Third","Fourth","Fifth"])
pd.Series([30, 35, 40], index=['2015 Sales', '2016 Sales', '2017 Sales'], name='Product A')

2015 Sales    30
2016 Sales    35
2017 Sales    40
Name: Product A, dtype: int64

### Dataframes

In [4]:
# Dataframe : can be seen as tables
df = pd.DataFrame({
    "Year":[2020,2021,2022],
    "Color":["Red","Green","Blue"],
    },
    index=["First","Second","Third"]
    )
df.reset_index()
df.set_index("Year")

,Color
Year,
2020,Red
2021,Green
2022,Blue


In [5]:
# quick df size/shape overview 
print(f"Features: \t{list(df.columns)}")
print(f"Size: {df.size} \t(rows * columns (if all fields are defined for each rows))")
print(f"Shape: {df.shape} \t(rows, cols)",end="\n\n") #(rows, cols)
df.info()

Features: 	['Year', 'Color']
Size: 6 	(rows * columns (if all fields are defined for each rows))
Shape: (3, 2) 	(rows, cols)

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, First to Third
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    3 non-null      int64 
 1   Color   3 non-null      object
dtypes: int64(1), object(1)
memory usage: 72.0+ bytes


## Importing data

In [6]:
#wines = pd.read_excel("file.xlsx")
#wines = pd.read_parquet("file.parquet") # less memory consument
wines = pd.read_csv("data/pandas_basics/winemag-data-130k-v2.csv")

In [7]:
wines.head(1)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia


## Indexing, selecting

In [8]:
wines[pd.isna(wines["taster_name"])].head(3)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
31,31,Italy,Merlot and Nero d'Avola form the base for this...,Calanìca Nero d'Avola-Merlot,86,NaN,Sicily & Sardinia,Sicilia,NaN,NaN,NaN,Duca di Salaparuta 2010 Calanìca Nero d'Avola-...,Red Blend,Duca di Salaparuta
32,32,Italy,"Part of the extended Calanìca series, this Gri...",Calanìca Grillo-Viognier,86,NaN,Sicily & Sardinia,Sicilia,NaN,NaN,NaN,Duca di Salaparuta 2011 Calanìca Grillo-Viogni...,White Blend,Duca di Salaparuta
33,33,US,"Rustic and dry, this has flavors of berries, c...",Puma Springs Vineyard,86,50.0,California,Dry Creek Valley,Sonoma,NaN,NaN,Envolve 2010 Puma Springs Vineyard Red (Dry Cr...,Red Blend,Envolve


In [9]:
# selecting a subset of features 
colsOfInterest = ["title","country","province","description","price","points","variety","winery","taster_name"]
# setting conditions on rows to be selected
rowsOfInterest = (wines["price"] < 15) & (wines["variety"] == "Pinot Gris")
rowsOfInterest = (wines["country"].isin(["Italy","France","US",""]))
# some 'taster_name' are of type NaN so they have to be replaced or rows has to be dropped to perform following command
rowsOfInterest = (wines["taster_name"].fillna("Unknown").str.startswith("Kerin"))


# df.loc[#row,#colums]
wines.loc[
    rowsOfInterest, #rows selected based on a condition
    colsOfInterest
    ].head(2)


# df.iloc[#rowIndex,#colIndex]
wines.iloc[0:3, [9,1,2,3]]

,taster_name,country,description,designation
0,Kerin O’Keefe,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco
1,Roger Voss,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos
2,Paul Gregutt,US,"Tart and snappy, the flavors of lime flesh and...",NaN


In [10]:
italianOrManyPoints = ((wines["country"] == "Italy") | (wines["points"] >= 90))

# sorting values
wines.loc[italianOrManyPoints,colsOfInterest].sort_values("country",ascending=False).head(1)

,title,country,province,description,price,points,variety,winery,taster_name
10532,Bouza 2013 Albariño (Montevideo),Uruguay,Montevideo,"With attractive, authentic aromas of lemon, le...",25.0,90,Albariño,Bouza,Michael Schachner


## Grouping and aggregation

In [11]:
# groupby(), value_counts() and size()
amountOfWinesPerCountry = wines.groupby("country")["country"].value_counts().head()


# aggregating wines per country using the mean of the 'points' and the max value of the 'price'.
wines.groupby("country").agg({"points":"mean","price":"max"}).sort_values("country",ascending=False).head(5)

,points,price
country,,
Uruguay,86.752294,130.0
Ukraine,84.071429,13.0
US,88.563720,2013.0
Turkey,88.088889,120.0
Switzerland,88.571429,160.0


In [12]:
countries = pd.Series(wines.loc[:,"country"],name="countries")

# value_counts() equivalent to the groupby().count()
numberOfWinesPerCountries = countries.value_counts()
winesFromAmerica = numberOfWinesPerCountries.get("US",0)

print(f"{(winesFromAmerica / wines.shape[0]) * 100:.2f} % of wines are from America")
numberOfWinesPerCountries.head(2)

41.94 % of wines are from America


countries
US        54504
France    22093
Name: count, dtype: int64

In [13]:
# selecting based on data availability 
wines.loc[wines["price"].isna()].count()
wines.loc[wines["price"].notnull()]
wines.loc[wines["price"].notna()].head(2)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


In [14]:
wines["country"].describe()

count     129908
unique        43
top           US
freq       54504
Name: country, dtype: object

In [15]:
# stats
print("Wine price stats:")
print(f"\tMean price: {wines["price"].mean()}")
print(f"\tMin price: {wines["price"].min()}")
print(f"\tMax price: {wines["price"].max()} (at index: {wines["price"].idxmax()})\n\n")

print(f"Wines per: {wines["country"].value_counts().head(3)}")
print(f"Possible values: {list(wines["country"].unique())}")

Wine price stats:
	Mean price: 35.363389129985535
	Min price: 4.0
	Max price: 3300.0 (at index: 80290)


Wines per: country
US        54504
France    22093
Italy     19540
Name: count, dtype: int64
Possible values: ['Italy', 'Portugal', 'US', 'Spain', 'France', 'Germany', 'Argentina', 'Chile', 'Australia', 'Austria', 'South Africa', 'New Zealand', 'Israel', 'Hungary', 'Greece', 'Romania', 'Mexico', 'Canada', nan, 'Turkey', 'Czech Republic', 'Slovenia', 'Luxembourg', 'Croatia', 'Georgia', 'Uruguay', 'England', 'Lebanon', 'Serbia', 'Brazil', 'Moldova', 'Morocco', 'Peru', 'India', 'Bulgaria', 'Cyprus', 'Armenia', 'Switzerland', 'Bosnia and Herzegovina', 'Ukraine', 'Slovakia', 'Macedonia', 'China', 'Egypt']


## Modifying rows : apply(), map(), broadcasting

In [16]:
# map(), apply() and broadcasting() : centering the 'points' 

points_mean = wines["points"].mean()

# broadcasting (the single mean value is converted to an array of same size as the 'points' column)
wines["points"] = wines["points"] - points_mean

def remean_points(row):
    row["points"] = row["points"] - points_mean
    return row

# using 'map' function
wines["points"].map(lambda p : p - points_mean).head(3)

# using 'apply' function (when using a custom method on each row)
wines["points"].apply(lambda p : p - points_mean).head(5)
wines.apply(remean_points,axis="columns").head(3)


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,-89.894276,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,-89.894276,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,-89.894276,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


In [17]:
wines.country + " - " + wines.region_1

0                     Italy - Etna
1                              NaN
2           US - Willamette Valley
3         US - Lake Michigan Shore
4           US - Willamette Valley
                    ...           
129966                         NaN
129967                 US - Oregon
129968             France - Alsace
129969             France - Alsace
129970             France - Alsace
Length: 129971, dtype: object

In [18]:
# getting wines with max points by country and province
wines.groupby(['country', 'province'])[colsOfInterest].apply(lambda df: df.loc[df.points.idxmax()]).head(3)

title  \
country   province                                                              
Argentina Mendoza Province  Bodega Catena Zapata 2006 Nicasia Vineyard Mal...   
          Other                            Colomé 2010 Reserva Malbec (Salta)   
Armenia   Armenia                 Van Ardi 2015 Estate Bottled Rosé (Armenia)   

                              country          province  \
country   province                                        
Argentina Mendoza Province  Argentina  Mendoza Province   
          Other             Argentina             Other   
Armenia   Armenia             Armenia           Armenia   

                                                                  description  \
country   province                                                              
Argentina Mendoza Province  If the color doesn't tell the full story, the ...   
          Other             Take note, this could be the best wine Colomé ...   
Armenia   Armenia           Deep salmon in color, this wine offers a bouqu...   

                            price    points variety                winery  \
country   province                                                          
Argentina Mendoza Province  120.0  8.552862  Malbec  Bodega Catena Zapata   
          Other              90.0  6.552862  Malbec                Colomé   
Armenia   Armenia            15.0 -0.447138    Rosé              Van Ardi   

                                  taster_name  
country   province                             
Argentina Mendoza Province  Michael Schachner  
          Other             Michael Schachner  
Armenia   Armenia               Mike DeSimone

## Data types and missing values

In [19]:
# wines.dtypes
# wines.price.dtype
wines.points.dtype
# wines.points.astype('int64')

dtype('float64')

In [20]:
# get all rows where the 'country' is NaN
# wines.loc[wines["country"].isna()].head(2) # way n°1
wines[pd.isnull(wines.country)].head(1) # way n°2

# fill not available values (NaN values)
wines["region_2"].fillna("Unknown")
# replace 
wines["taster_twitter_handle"].replace("@kerinokeefe", "@kerino")

0             @kerino
1          @vossroger
2         @paulgwine 
3                 NaN
4         @paulgwine 
             ...     
129966            NaN
129967    @paulgwine 
129968     @vossroger
129969     @vossroger
129970     @vossroger
Name: taster_twitter_handle, Length: 129971, dtype: object

## Renaming and combining

In [21]:
# rename columns, indexes or axis  
wines.rename(columns={'points':'score'}, index={0:'firstEntry', 1: 'secondEntry'}).head(1)
wines.rename_axis("wines", axis='rows').rename_axis("features", axis='columns').head(1)

features,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
wines,,,,,,,,,,,,,,
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,-1.447138,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia


In [22]:
# combine dataframes/series
left = pd.DataFrame({
    "temperatures":[30,28,32,29]
    })

right = pd.DataFrame({
    "temperatures":[20,21,22,19]
    })


pd.concat([left,right],axis='columns')

,temperatures,temperatures
0,30,20
1,28,21
2,32,22
3,29,19


In [23]:
# combining based on a common index
left.join(right,lsuffix='_CAN',rsuffix='_UK')

,temperatures_CAN,temperatures_UK
0,30,20
1,28,21
2,32,22
3,29,19
